In [ ]:
#Set OpenVINO environment if it is not set. (Jupyter demands this to be set before running the jupyter)
!"C:\Intel\computer_vision_sdk\bin\setupvars.bat"

# OpenVINO Vehicle Analytics Demo

In [ ]:
#See the demo details
!monitor.exe -help

In [ ]:
#Download the sample video having cars entering and exiting
!curl -L -o car-detection.mp4 "https://github.com/intel-iot-devkit/sample-videos/raw/master/car-detection.mp4"

In [ ]:
#Install Mosquitto - an Open Source MQTT broker - Eclipse, if it is not installed in the system
#!curl -L -o mosquitto-1.5.7-install-windows-x64.exe "https://mosquitto.org/files/binary/win64/mosquitto-1.5.7-install-windows-x64.exe"

In [ ]:
#!mosquitto-1.5.7-install-windows-x64.exe
#Add "C:\Program Files\mosquitto" in path variable
#Restart the jupyter notebook after installation of mosquitto
#Run mosquitto in one command prompt
#Run below mosquitto_sub in another command prompt

#Vehicle Analytics on CPU - run the following command in the separate command prompt by removing # symbol
#Below command is to show the mqtt subscribre message comming from the 'parking/counter' topic 
#from vehicle analytics application. Publishing & Subscribing host is 'test.mosquitto.org'
***
<b>mosquitto_sub -v -t parking/counter -h test.mosquitto.org</b>
***

In [ ]:
#Run the demo
!start "" cmd /C monitor.exe -m="C:/Intel/computer_vision_sdk/deployment_tools/intel_models/pedestrian-and-vehicle-detector-adas-0001/FP32/pedestrian-and-vehicle-detector-adas-0001.bin" -c="C:/Intel/computer_vision_sdk/deployment_tools/intel_models/pedestrian-and-vehicle-detector-adas-0001/FP32/pedestrian-and-vehicle-detector-adas-0001.xml" -i="car-detection.mp4" -cc=0.7 -e="b"

# OpenVINO Retail Analytics Demo

In [ ]:
!git clone https://github.com/intel-iot-devkit/store-traffic-monitor-python.git

In [ ]:
%cd store-traffic-monitor-python

In [ ]:
#Download the sample video
!start "" cmd /K python video_downloader.py

In [ ]:
#Download the caffe model to resources folder
!python C:\Intel\computer_vision_sdk\deployment_tools\model_downloader\downloader.py --name mobilenet-ssd -o resources

In [ ]:
#Convert Caffe model to Intel OpenVINO FP32 IR format (.xml & .bin)
!python C:\Intel\computer_vision_sdk\deployment_tools\model_optimizer\mo_caffe.py --input_model resources\object_detection\common\mobilenet-ssd\caffe\mobilenet-ssd.caffemodel -o resources --mean_values [103.94,116.78,123.68] --scale 58.8235

In [ ]:
#Run the demo by properly providing the relevant directories
#Retail Analytics on CPU
#Verify the Line no. 331     n, c, h, w = net.inputs[input_blob].shape
#if shape is not added to the source code on this line, please add it
#If you get an error from OpenH264 Video Codec provided by Cisco Systems, Inc. 
#then download http://ciscobinary.openh264.org/openh264-1.6.0-win64msvc.dll.bz2, extract and copy it to system32 folder
!start "" cmd /K python store-traffic-monitor.py -m resources\mobilenet-ssd.xml -l resources\labels.txt -e "%HOMEDRIVE%%HOMEPATH%\Documents\Intel\OpenVINO\inference_engine_samples_2017\intel64\Release\cpu_extension.dll" -d CPU

In [ ]:
#Retail Analytics on GPU
!start "" python store-traffic-monitor.py -m resources/mobilenet-ssd.xml -l resources/labels.txt -d GPU

Movidius driver:-
If not done before or if it is not working, install the Movidius driver by right click and install command -
C:\Intel\computer_vision_sdk\deployment_tools\inference_engine\external\MovidiusDriver\Movidius_VSC_Device.inf

In [ ]:
#Convert Caffe model to Intel OpenVINO FP16 IR format (.xml & .bin) for inference on Movidius VPU
!python C:\Intel\computer_vision_sdk\deployment_tools\model_optimizer\mo_caffe.py --input_model resources\object_detection\common\mobilenet-ssd\caffe\mobilenet-ssd.caffemodel -o resources --mean_values [103.94,116.78,123.68] --scale 58.8235 --data_type FP16 --model_name mobilenet-ssd-fp16

In [ ]:
#Run the demo by properly providing the relevant directories
#Retail Analytics on Movidius VPU
!start "" cmd /C python store-traffic-monitor.py -m resources/mobilenet-ssd-fp16.xml -l resources/labels.txt -d MYRIAD